<a href="https://colab.research.google.com/github/matzmtok/AdvancedPytorch/blob/main/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%B3%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ファインチューニングの実装

ファインチューニングを使用して、手元にある少量のデータを学習させて、オリジナルの画像分類モデルを構築する。

アリとハチの画像を分類するモデルを学習させる。

1. PytorchでGPUを使用する実装コードをかけるようになる
2. 最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる
3. 学習したネットワークを保存・ロードできるようになる

## ファインチューニング

ファインチューニングとは、学習済みモデルをベースに出力層などを変更したモデルを構築し、自前のデータでニューラルネットワーク・モデルの結合パラメータを学習させる手法です。
転移学習とは異なり、出力層および出力層に近い部分だけでなく、全層のパラメータを再学習させます。ただし、入力層に近い部分のパラメータの学習率は小さく設定し、出力層に近いパラメータの学習率は大きく設定することが一般的。

ファインチューニングは転移学習同様に、学習済みモデルを利用するため、自前のデータが少量でも性能の良いディープラーニングを実現しやすい。

## フォルダ準備と事前準備

## DatasetとDataLoaderを作成



In [1]:
import glob
import os
import os.path as osp
import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data as data
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class ImageTransform():
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train': transforms.Compose([
          transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
          transforms.Resize(resize),
          transforms.CenterCrop(resize),
          transforms.ToTensor(),
          transforms.Normalize(mean, std)
        ])
    }

  def __call__(self, img, phase='train'):
    """
    Parameters
    ----------
    phase : 'train' or 'val'
    引数に指定したimgに対してtransformを適用する
    """
    return self.data_transform[phase](img)

def make_datapath_list(phase="train"):
  """
  対象フォルダ内の画像のリストを生成する
  """
  rootpath = "/content/drive/MyDrive/datasets/hymenoptera_data"
  target_path = os.path.join(rootpath, phase ,"**/*.jpg")
  print(f"target path : {target_path}")
  path_list = []

  for path in glob.glob(target_path):
    path_list.append(path)

  return path_list

class HymenopteraDataset(data.Dataset):
  """
  アリとハチの画像のDatastクラス
  """
  def __init__(self, file_list, transform=None, phase='train'):
    self.file_list = file_list
    self.transform = transform
    self.phase = phase

  def __len__(self):
    """
    画像の枚数を返す
    """
    return len(self.file_list)

  def __getitem__(self, index):
    """
    前処理をした画像のTensor形式のデータトラベルを取得
    """
    img_path = self.file_list[index]
    img = Image.open(img_path)

    img_transformed = self.transform(img, self.phase)

    train_idx = len("/content/drive/MyDrive/datasets/hymenoptera_data/train/")
    val_idx = len("/content/drive/MyDrive/datasets/hymenoptera_data/val/")
    if self.phase == "train":
      label = img_path[train_idx:train_idx+4]
    elif self.phase == "val":
      label = img_path[val_idx:val_idx+4]

    if label == "ants":
      label = 0
    elif label == "bees":
      label = 1
    
    return img_transformed, label


  

In [4]:
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.299, 0.224, 0.225)

train_dataset = HymenopteraDataset(train_list, transform=ImageTransform(size, mean, std), phase="train")
val_dataset = HymenopteraDataset(val_list, transform=ImageTransform(size, mean, std),phase="val")

batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

target path : /content/drive/MyDrive/datasets/hymenoptera_data/train/**/*.jpg
target path : /content/drive/MyDrive/datasets/hymenoptera_data/val/**/*.jpg


## ネットワークモデルの作成

出力層を1000クラスからアリとハチの2クラスとなるように付け替える

In [5]:
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

print(net)

# 出力層の付替
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

#test
net.train()

print(net.named_parameters())


print('network is configured! It\'s ready to use.')


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 損失関数を定義

In [6]:
criterion = nn.CrossEntropyLoss()



## 最適化手法を設定

ファインチューニングでは、最適化手法の設定部分が転移学習と異なる。全層のパラメータを学習できるようにoptimizerを設定する。

各層ごとに学習率を変えられるようにパラメータを設定する。
VGG-16の前半のfeatureモジュールのパラメータ変数update_param_names_1に、後半の全層結合層のclassifierモジュールのうち、最初の2つの全層結合のパラメータを変数update_param_names_2に、そして付け替えた最後の全層結合層のパラメータ変数をupdate_param_names_3に格納し、それぞれを異なる学習率を適用する。


In [7]:
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

update_params_names_1 = ["features"]
update_params_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_params_names_3 = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
  if update_params_names_1[0] in name:
    param.requires_grad = True
    params_to_update_1.append(param)
    print("params_to_update_1に格納：", name)

  elif name in  update_params_names_2:
    param.requires_grad = True
    params_to_update_2.append(param)
    print("params_to_update_2に格納：", name)

  elif name in update_params_names_3:
    param.requires_grad = True
    params_to_update_3.append(param)
    print("params_to_update_3に格納：", name)
    
  else:
    param.requires_grad = False

optimizer = optim.SGD([
  {'params': params_to_update_1, 'lr': 1e-4},
  {'params': params_to_update_2, 'lr': 5e-4},
  {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

params_to_update_1に格納： features.0.weight
params_to_update_1に格納： features.0.bias
params_to_update_1に格納： features.2.weight
params_to_update_1に格納： features.2.bias
params_to_update_1に格納： features.5.weight
params_to_update_1に格納： features.5.bias
params_to_update_1に格納： features.7.weight
params_to_update_1に格納： features.7.bias
params_to_update_1に格納： features.10.weight
params_to_update_1に格納： features.10.bias
params_to_update_1に格納： features.12.weight
params_to_update_1に格納： features.12.bias
params_to_update_1に格納： features.14.weight
params_to_update_1に格納： features.14.bias
params_to_update_1に格納： features.17.weight
params_to_update_1に格納： features.17.bias
params_to_update_1に格納： features.19.weight
params_to_update_1に格納： features.19.bias
params_to_update_1に格納： features.21.weight
params_to_update_1に格納： features.21.bias
params_to_update_1に格納： features.24.weight
params_to_update_1に格納： features.24.bias
params_to_update_1に格納： features.26.weight
params_to_update_1に格納： features.26.bias
params_to_update_1に格納： f

In [8]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("使用デバイス：", device)

  net.to(device)

  torch.backends.cudnn.benchmark = True

  for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{ num_epochs}")
    print('------------------------------')

    for phase in ['train', 'val']:
      if phase =='train':
        net.train()
      else:
        net.eval()

      epoch_loss = 0.0
      epoch_corrects = 0

      # epoch 0 は学習しない。
      # 未学習時の性能を確認するため
      if (epoch == 0) and (phase == 'train'):
        continue
      
      for inputs, labels in tqdm(dataloaders_dict[phase]):
        inputs = inputs.to(device)
        print(labels)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(inputs) # 推論実行
          loss = criterion(outputs, labels) # 損失計算
          _, preds = torch.max(outputs, 1) # ラベルを予測

          # 逆伝搬
          if phase == 'train':
            loss.backward()
            optimizer.step()

          # 計算したロスを合計
          epoch_loss += loss.item() * inputs.size(0)
          epoch_corrects += torch.sum(preds == labels.data)
      
      # epochごとのLossと成果率を表示
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

      print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")



In [9]:
num_epochs = 2

train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
Epoch 1/2
------------------------------


  0%|          | 0/5 [00:00<?, ?it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])


 40%|████      | 2/5 [00:23<00:34, 11.58s/it]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])


...

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


 80%|████████  | 4/5 [00:01<00:00,  2.01it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


100%|██████████| 5/5 [00:02<00:00,  2.18it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
val Loss: 0.2451 Acc: 0.9085


## 学習したネットワークを保存・ロード

ネットワークモデルの変数net に対して、.state_dict()でパラメータを辞書型変数として取り出し、torch.save()で保存する。



In [10]:
save_path = "/content/drive/MyDrive/datasets/hymenoptera_data/artifact/weights_fine_tuning.pth"
torch.save(net.state_dict(), save_path)

### データのロード




In [ ]:
load_path = "/content/drive/MyDrive/datasets/hymenoptera_data/artifact"

load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# gpuで保存した重みをCPUにロードする場合
# load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
# net.load_state_dict(load_weighs)